<h1>Crash Severity Prediction Model</h1>

Let's build a model to predict the severity of a crash.

In [1]:
# Import all required libraries

import pandas as pd 
from glob import glob
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [2]:
# We need a list containing all of the columns we'll use for our model

filtered_columns =  [
 'Schl_Bus_Fl',
 'Rr_Relat_Fl',
 'Active_School_Zone_Fl',
 'Crash_Date',
 'Crash_Time',
 'Toll_Road_Fl',
 'Crash_Speed_Limit',
 'Road_Constr_Zone_Fl',
 'Road_Constr_Zone_Wrkr_Fl',
 'At_Intrsct_Fl',
 'Wthr_Cond_ID',
 'Light_Cond_ID',
 'Road_Type_ID',
 'Road_Algn_ID',
 'Surf_Cond_ID',
 'Intrsct_Relat_ID',
 'FHE_Collsn_ID',
 'Obj_Struck_ID',
 'Othr_Factr_ID',
 'Road_Part_Adj_ID',
 'Road_Cls_ID',
 'Road_Relat_ID',
 'Phys_Featr_1_ID',
 'Phys_Featr_2_ID',
 'Pop_Group_ID',
 'Day_of_Week',
 'Base_Type_ID',
 'Surf_Type_ID',
 'Adt_Adj_Curnt_Amt',
 'Trk_Aadt_Pct',
 'Curve_Type_ID',
 'Curve_Lngth',
 'Cd_Degr',
 'Delta_Left_Right_ID',
 'Dd_Degr',
 'WDCode_ID',
 'Crash_Sev_ID']

In [3]:
# We also need a list of just the predictor columns

predictor_columns =  [
 'Schl_Bus_Fl',
 'Rr_Relat_Fl',
 'Active_School_Zone_Fl',
 'Crash_Date',
 'Crash_Time',
 'Toll_Road_Fl',
 'Crash_Speed_Limit',
 'Road_Constr_Zone_Fl',
 'Road_Constr_Zone_Wrkr_Fl',
 'At_Intrsct_Fl',
 'Wthr_Cond_ID',
 'Light_Cond_ID',
 'Road_Type_ID',
 'Road_Algn_ID',
 'Surf_Cond_ID',
 'Intrsct_Relat_ID',
 'FHE_Collsn_ID',
 'Obj_Struck_ID',
 'Othr_Factr_ID',
 'Road_Part_Adj_ID',
 'Road_Cls_ID',
 'Road_Relat_ID',
 'Phys_Featr_1_ID',
 'Phys_Featr_2_ID',
 'Pop_Group_ID',
 'Day_of_Week',
 'Base_Type_ID',
 'Surf_Type_ID',
 'Adt_Adj_Curnt_Amt',
 'Trk_Aadt_Pct',
 'Curve_Type_ID',
 'Curve_Lngth',
 'Cd_Degr',
 'Delta_Left_Right_ID',
 'Dd_Degr',
 'WDCode_ID']

In [4]:
# Import the data to a pandas data frame

root = "extract_public_2018_20200825003448226_69851_20190101-20191231Texas/"
files = glob(root + '*/*_crash_*.csv')
get_df = lambda f: pd.read_csv(f)
dfdict = {f: pd.read_csv(f) for f in files}
df = pd.concat(dfdict)
df.reset_index()

,level_0,level_1,Crash_ID,Crash_Fatal_Fl,Cmv_Involv_Fl,Schl_Bus_Fl,Rr_Relat_Fl,Medical_Advisory_Fl,Amend_Supp_Fl,Active_School_Zone_Fl,...,Nonincap_Injry_Cnt,Poss_Injry_Cnt,Non_Injry_Cnt,Unkn_Injry_Cnt,Tot_Injry_Cnt,Death_Cnt,MPO_ID,Investigat_Service_ID,Investigat_DA_ID,Investigator_Narrative
0,extract_public_2018_20200825003448226_69851_20...,0,15657177,N,N,N,N,N,Y,N,...,0,0,0,1,0,0,NaN,35.0,NaN,NaN
1,extract_public_2018_20200825003448226_69851_20...,1,16406486,N,N,N,N,N,Y,N,...,0,0,1,1,0,0,NaN,23.0,NaN,NaN
2,extract_public_2018_20200825003448226_69851_20...,2,16473665,N,N,N,N,N,Y,N,...,1,0,1,0,1,0,NaN,35.0,NaN,NaN
3,extract_public_2018_20200825003448226_69851_20...,3,16871051,N,Y,N,N,N,Y,N,...,0,0,2,0,0,0,282.0,NaN,NaN,NaN
4,extract_public_2018_20200825003448226_69851_20...,4,16995273,N,N,N,N,N,Y,N,...,0,1,6,0,1,0,15.0,53.0,61.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648332,extract_public_2018_20200825003448226_69851_20...,54572,17777131,N,N,N,N,N,N,N,...,0,0,0,2,0,0,NaN,29.0,14.0,NaN
648333,extract_public_2018_20200825003448226_69851_20...,54573,17779640,N,N,N,N,N,N,N,...,0,0,2,0,0,0,15.0,NaN,NaN,NaN
648334,extract_public_2018_20200825003448226_69851_20...,54574,17783015,N,N,N,N,N,N,N,...,0,0,0,1,0,0,NaN,411.0,NaN,NaN
648335,extract_public_2018_20200825003448226_69851_20...,54575,17801355,N,N,N,N,N,N,N,...,0,0,1,0,0,0,15.0,NaN,NaN,NaN


In [5]:
# Filter the imported data based on our selected features.

df_filtered = df[filtered_columns]
#df_filtered = df

In [6]:
# Drop all rows where there is no crash severity ID listed. This is our target column, so these rows are useless

df_filtered = df_filtered[df_filtered.Crash_Sev_ID != 0]

In [7]:
# Let's also drop all rows where the speed limit is -1, which is a effectively a "NaN" value.
df_filtered = df_filtered[df_filtered['Crash_Speed_Limit']!=-1]

# df_filtered.dropna(inplace=True)

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))

df_filtered['Crash_Time'] = pd.to_datetime(df_filtered['Crash_Time']).astype('int64')

values = df_filtered['Crash_Time'].values
values = values.reshape((len(values), 1))

df_filtered['NORMTIME'] = scaler.fit_transform(values)

df_filtered.drop('Crash_Time', axis=1, inplace=True)
df_filtered.rename(columns={'NORMTIME' : "Crash_Time"}, inplace=True)

In [ ]:
df_filtered['Crash_Date'] = pd.to_datetime(df_filtered['Crash_Date']).astype('int64')

values = df_filtered['Crash_Date'].values
values = values.reshape((len(values), 1))

df_filtered['NORMDATE'] = scaler.fit_transform(values)

df_filtered.drop('Crash_Date', axis=1, inplace=True)
df_filtered.rename(columns={'NORMDATE' : "Crash_Date"}, inplace=True)

In [ ]:
df_filtered.Day_of_Week.replace(to_replace=r'^MON$', value=1, regex=True, inplace=True)
df_filtered.Day_of_Week.replace(to_replace=r'^TUE$', value=2, regex=True, inplace=True)
df_filtered.Day_of_Week.replace(to_replace=r'^WED$', value=3, regex=True, inplace=True)
df_filtered.Day_of_Week.replace(to_replace=r'^THU$', value=4, regex=True, inplace=True)
df_filtered.Day_of_Week.replace(to_replace=r'^FRI$', value=5, regex=True, inplace=True)
df_filtered.Day_of_Week.replace(to_replace=r'^SAT$', value=6, regex=True, inplace=True)
df_filtered.Day_of_Week.replace(to_replace=r'^SUN$', value=7, regex=True, inplace=True)

values = df_filtered['Day_of_Week'].values
values = values.reshape((len(values), 1))

df_filtered['NORMDAY'] = scaler.fit_transform(values)

df_filtered.drop('Day_of_Week', axis=1, inplace=True)
df_filtered.rename(columns={'NORMDAY' : "Day_of_Week"}, inplace=True)

In [ ]:
# We need to normalize our text values - y/n to 1/0, and the days of the week normalized to values between 1 and 7.

df_filtered.replace(to_replace=r'^N$', value=0, regex=True, inplace=True)
df_filtered.replace(to_replace=r'^Y$', value=1, regex=True, inplace=True)

<h2>Balance the data</h2>

To avoid overfitting for a given target value, let's make sure the data is balanced.

In [ ]:
# Plot the crash severity IDs

sns.countplot(x='Crash_Sev_ID', data=df_filtered)
plt.xticks(rotation=90)

This is HEAVILY skewed towards 5, no injury. If we try to train as is, his is going to result in a model that predicts 5 more often than not, just by virtue of it appear most often in the data. 

Let's avoid this problem by balancing the data.

In [ ]:
count_class_1, count_class_2, count_class_3, count_class_4, count_class_5 = df_filtered.Crash_Sev_ID.value_counts()
df_class_1 = df_filtered[df_filtered['Crash_Sev_ID'] == 1]
df_class_2 = df_filtered[df_filtered['Crash_Sev_ID'] == 2]
df_class_3 = df_filtered[df_filtered['Crash_Sev_ID'] == 3]
df_class_4 = df_filtered[df_filtered['Crash_Sev_ID'] == 4]
df_class_5 = df_filtered[df_filtered['Crash_Sev_ID'] == 5]

In [ ]:
df_class_5_under = df_class_5.sample(count_class_5)
df_class_3_under = df_class_3.sample(count_class_5)
df_class_2_under = df_class_2.sample(count_class_5)
df_class_1_under = df_class_1.sample(count_class_5)

In [ ]:
df_test_under = pd.concat([df_class_5_under, df_class_3_under, df_class_2_under, df_class_1_under, df_class_4], axis=0)
df_test_under.shape

In [ ]:
print('Random under-sampling:')
print(df_test_under.Crash_Sev_ID.value_counts())
df_test_under.Crash_Sev_ID.value_counts().plot(kind='bar', title='Count (target)')

In [ ]:
df_class_5_over = df_class_5.sample(count_class_1, replace=True)
df_class_4_over = df_class_4.sample(count_class_1, replace=True)
df_class_3_over = df_class_3.sample(count_class_1, replace=True)
df_class_2_over = df_class_2.sample(count_class_1, replace=True)
df_class_1_over = df_class_1.sample(count_class_1, replace=True)

In [ ]:
df_test_over = pd.concat([df_class_4_over, df_class_3_over, df_class_2_over, df_class_1_over, df_class_5], axis=0)
df_test_over.shape

In [ ]:
print('Random over-sampling:')
print(df_test_over.Crash_Sev_ID.value_counts())
df_test_over.Crash_Sev_ID.value_counts().plot(kind='bar', title='Count (target)')

We now have two datasets that are much more balanced. But because of the sampling methods, we may run into issues down the line. We'll test both frames and compare the resulting models to see which is more useful for our purposes.

<h2>Decision Tree</h2>

In [ ]:
X = df_test_over[predictor_columns].values
y = df_test_over["Crash_Sev_ID"]
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.2, random_state=3)
crashTree = DecisionTreeClassifier(criterion="entropy", max_depth = 3)
crashTree.fit(X_trainset,y_trainset)
predTree = crashTree.predict(X_testset)

In [ ]:
accuracy_score(y_testset, predTree)

In [ ]:
recall_score(y_testset, predTree, average=None)

In [ ]:
precision_score(y_testset, predTree, average=None)

In [ ]:
f1_score(y_testset, predTree, average=None)

In [ ]:
conf_mat = confusion_matrix(y_true=y_testset, y_pred=predTree)
print('Confusion matrix:\n', conf_mat)

labels = ['1', '2', '3', '4', '5']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()

In [ ]:
X = df_test_under[predictor_columns].values
y = df_test_under["Crash_Sev_ID"]
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.2, random_state=3)
crashTree = DecisionTreeClassifier(criterion="entropy", max_depth = 3)
crashTree.fit(X_trainset,y_trainset)
predTree = crashTree.predict(X_testset)

In [ ]:
accuracy_score(y_testset, predTree)

In [ ]:
recall_score(y_testset, predTree, average=None)

In [ ]:
precision_score(y_testset, predTree, average=None)

In [ ]:
f1_score(y_testset, predTree, average=None)

In [ ]:
conf_mat = confusion_matrix(y_true=y_testset, y_pred=predTree)
print('Confusion matrix:\n', conf_mat)

labels = ['1', '2', '3', '4', '5']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()

We can see here that neither model is performing particularly well, but adjusting the max_depth may return better results. Let's find the best value for max_depth.

In [ ]:
X = df_test_over[predictor_columns].values
y = df_test_over["Crash_Sev_ID"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

max_depths = np.linspace(1, 60, 60, endpoint=True)
train_results = []

for max_depth in max_depths:
   dt = DecisionTreeClassifier(criterion="entropy", max_depth=max_depth)
   dt.fit(x_train, y_train)
   train_pred = dt.predict(x_test)
   train_pred_score = metrics.accuracy_score(y_test, train_pred)
   train_results.append(train_pred_score)

from matplotlib.legend_handler import HandlerLine2D
line1 = plt.plot(max_depths, train_results)
plt.show()

In [ ]:
X = df_test_under[predictor_columns].values
y = df_test_under["Crash_Sev_ID"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

max_depths = np.linspace(1, 60, 60, endpoint=True)
train_results = []

for max_depth in max_depths:
   dt = DecisionTreeClassifier(criterion="entropy", max_depth=max_depth)
   dt.fit(x_train, y_train)
   train_pred = dt.predict(x_test)
   train_pred_score = metrics.accuracy_score(y_test, train_pred)
   train_results.append(train_pred_score)

from matplotlib.legend_handler import HandlerLine2D
line1 = plt.plot(max_depths, train_results)
plt.show()

The under sampled data maxes out at 34% accuracy, which is not going to be usable for a model. But the oversampled data can get over 90% with a max_depth of around 35 before we start leveling off. We'll use the oversampled data.

In [ ]:
X = df_test_over[predictor_columns].values
y = df_test_over["Crash_Sev_ID"]
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.2, random_state=3)
crashTree = DecisionTreeClassifier(criterion="entropy", max_depth = 35)
crashTree.fit(X_trainset,y_trainset)
predTree = crashTree.predict(X_testset)

In [ ]:
accuracy_score(y_testset, predTree)

In [ ]:
recall_score(y_testset, predTree, average=None)

In [ ]:
precision_score(y_testset, predTree, average=None)

In [ ]:
f1_score(y_testset, predTree, average=None)

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_testset, predTree)
CM = CM / CM.astype(np.float).sum(axis=1)
labels = ['1', '2', '3', '4', '5']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(CM, cmap=plt.cm.Reds)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()

This looks really solid.  Recall is the model metric we use to select our best model when there is a high cost associated with False Negative. We have a recall of 1 for the value of 4, fatal accidents. We cannot afford to miss this prediction, so this may be a good model.

<h2>Random Forest</h2>

In [ ]:
X = df_test_over[predictor_columns].values
y = df_test_over["Crash_Sev_ID"]
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.2, random_state=3)

clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_trainset,y_trainset)
y_hat=clf.predict(X_testset)

In [ ]:
accuracy_score(y_testset, y_hat)

In [ ]:
recall_score(y_testset, y_hat, average=None)

In [ ]:
precision_score(y_testset, y_hat, average=None)

In [ ]:
f1_score(y_testset, y_hat, average=None)

In [ ]:
CM = confusion_matrix(y_testset, y_hat)
CM = CM / CM.astype(np.float).sum(axis=1)
labels = ['1', '2', '3', '4', '5']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(CM, cmap=plt.cm.Reds)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()

This is already miles ahead of where we were with the decision tree classifier. 

Let's take a deeper look at the features to see if we can improve the model any further. 

In [ ]:
feature_imp = pd.Series(clf.feature_importances_,index=df_test_over[predictor_columns].columns).sort_values(ascending=False)
feature_imp

In [ ]:
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

Some of these features are next to useless.

In [ ]:
new_filter = ['Crash_Time',
'Day_of_Week',
'Crash_Speed_Limit',
'FHE_Collsn_ID',
'Pop_Group_ID',
'Surf_Type_ID',
'Othr_Factr_ID',
'Obj_Struck_ID',
'Road_Cls_ID',
'Wthr_Cond_ID',
'Light_Cond_ID',
'Crash_Sev_ID']

new_predictors = ['Crash_Time',
'Day_of_Week',
'Crash_Speed_Limit',
'FHE_Collsn_ID',
'Pop_Group_ID',
'Surf_Type_ID',
'Othr_Factr_ID',
'Obj_Struck_ID',
'Road_Cls_ID',
'Wthr_Cond_ID',
'Light_Cond_ID']

In [ ]:
X = df_test_over[new_predictors].values
y = df_test_over["Crash_Sev_ID"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [ ]:
max_depths = np.linspace(1, 60, 60, endpoint=True)
train_results = []

for max_depth in max_depths:
   dt = DecisionTreeClassifier(criterion="entropy", max_depth=max_depth)
   dt.fit(x_train, y_train)
   train_pred = dt.predict(x_test)
   train_pred_score = metrics.accuracy_score(y_test, train_pred)
   train_results.append(train_pred_score)

from matplotlib.legend_handler import HandlerLine2D
line1 = plt.plot(max_depths, train_results)
plt.show()

In [ ]:
crashTree = DecisionTreeClassifier(criterion="entropy", max_depth = 34)
crashTree.fit(x_train,y_train)
y_hat = crashTree.predict(x_test)

In [ ]:
accuracy_score(y_test, y_hat)

In [ ]:
recall_score(y_test, y_hat, average=None)

In [ ]:
precision_score(y_test, y_hat, average=None)

In [ ]:
f1_score(y_test, y_hat, average=None)

In [ ]:
CM = confusion_matrix(y_test, y_hat)
CM = CM / CM.astype(np.float).sum(axis=1)
labels = ['1', '2', '3', '4', '5']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(CM, cmap=plt.cm.Reds)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()

In [ ]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(x_train,y_train)
y_hat=clf.predict(x_test)

In [ ]:
accuracy_score(y_test, y_hat)

In [ ]:
recall_score(y_test, y_hat, average=None)

In [ ]:
precision_score(y_test, y_hat, average=None)

In [ ]:
f1_score(y_test, y_hat, average=None)

In [ ]:
CM = confusion_matrix(y_test, y_hat)
CM = CM / CM.astype(np.float).sum(axis=1)
labels = ['1', '2', '3', '4', '5']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(CM, cmap=plt.cm.Reds)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()

We lost accuracy on the decision tree by dropping those rows, 91.13% down to 91.07%.

We also slightly lost accuracy on the random forest, but we got our best F1 score yet.

I am comfortable with this model.

<h2>Testing the model</h2>

To see if the model can perform consistently, let's test the model against data that the model has never seen before. I've downloaded the crash data from 2016. Let's put it through the same normalization pipeline first, and run them through the model.

In [ ]:
test_df = pd.read_csv('crash_2016.csv')
test_df_filtered = test_df[new_filter]
test_df_filtered = test_df_filtered[test_df_filtered.Crash_Sev_ID != 0]
test_df_filtered = test_df_filtered.replace(to_replace=r'^N$', value=0, regex=True)
test_df_filtered = test_df_filtered.replace(to_replace=r'^Y$', value=1, regex=True)
test_df_filtered = test_df_filtered.replace(to_replace=r'^MON$', value=1, regex=True)
test_df_filtered = test_df_filtered.replace(to_replace=r'^TUE$', value=2, regex=True)
test_df_filtered = test_df_filtered.replace(to_replace=r'^WED$', value=3, regex=True)
test_df_filtered = test_df_filtered.replace(to_replace=r'^THU$', value=4, regex=True)
test_df_filtered = test_df_filtered.replace(to_replace=r'^FRI$', value=5, regex=True)
test_df_filtered = test_df_filtered.replace(to_replace=r'^SAT$', value=6, regex=True)
test_df_filtered = test_df_filtered.replace(to_replace=r'^SUN$', value=7, regex=True)
test_df_filtered['Crash_Time'] = pd.to_datetime(test_df_filtered['Crash_Time']).astype('int64')
max_a = test_df_filtered.Crash_Time.max()
min_a = test_df_filtered.Crash_Time.min()
min_norm = -1
max_norm =1
test_df_filtered['NORMTIME'] = (test_df_filtered.Crash_Time- min_a) *(max_norm - min_norm) / (max_a-min_a) + min_norm
test_df_filtered.drop('Crash_Time', axis=1, inplace=True)
test_df_filtered.rename(columns={'NORMTIME' : "Crash_Time"}, inplace=True)
test_df_filtered = test_df_filtered[test_df_filtered['Crash_Speed_Limit']!=-1]
test_df_filtered.dropna(inplace=True)

In [ ]:
X = test_df_filtered[new_predictors].values
y = test_df_filtered["Crash_Sev_ID"]

In [ ]:
y_hat = crashTree.predict(X)

In [ ]:
accuracy_score(y, y_hat)

In [ ]:
recall_score(y, y_hat, average=None)

In [ ]:
precision_score(y, y_hat, average=None)

In [ ]:
f1_score(y, y_hat, average=None)

In [ ]:
CM = confusion_matrix(y, y_hat)
CM = CM / CM.astype(np.float).sum(axis=1)
labels = ['1', '2', '3', '4', '5']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(CM, cmap=plt.cm.Reds)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()

In [ ]:
y_hat = clf.predict(X)

In [ ]:
accuracy_score(y, y_hat)

In [ ]:
recall_score(y, y_hat, average=None)

In [ ]:
precision_score(y, y_hat, average=None)

In [ ]:
f1_score(y, y_hat, average=None)